In [2]:
import pandas as pd
import numpy as np
from plotnine import *

%load_ext lab_black

In [5]:
data = pd.read_csv(
    "https://raw.githubusercontent.com/MIDS-at-Duke/estimating-impact-of-opioids-2020-purlple-team/master/20_intermediate_files/2006-2012_prescription_finalStates.csv?token=ARFW6V3TUTPXYTVWKGTZNJ27WSK2W"
)
pop = pd.read_csv(
    "https://raw.githubusercontent.com/MIDS-at-Duke/estimating-impact-of-opioids-2020-purlple-team/ssloate/20_intermediate_files/countypopulations_clean.csv?token=ARFW6V3JPVHSDMBMO7HI2I27WSK7S"
)
fips = pd.read_excel(
    "/Users/samsloate/Desktop/Data_Science/Opioids_Project/estimating-impact-of-opioids-2020-purlple-team/00_source/fips_codes.xlsx"
)

In [6]:
data = data.copy()
pop = pop.copy()
fips = fips.copy()
data = data.rename(
    columns={"BUYER_STATE": "State", "BUYER_COUNTY": "County", "YEAR": "Year"}
)

In [7]:
# change County columns to sentence case and add the word county at the end
data["County"] = data["County"].str.capitalize()

# add County to end
data["County"] = data["County"] = data["County"] + " County"

# change fips column name to match County
fips = fips.rename(columns={"Full Name": "County"})

# drop _merge column in pop
pop = pop.drop(columns="_merge")

In [8]:
# drop Alaska (Nick said to) and Louisiana (we know they aren't a match based on other graphing)
data = data[data["State"] != "AK"]
data = data[data["State"] != "LA"]
fips = fips[fips["State"] != "AK"]
fips = fips[fips["State"] != "LA"]


# make new fips dataset for only the states we have in the shipment dataset
fips_cut = fips.loc[fips["State"].isin(list(data["State"].unique()))]

# check it's the right length / has all states
assert len(fips_cut.State.unique()) == len(list(data["State"].unique()))

In [9]:
len(list(data["State"].unique()))

6

In [10]:
# fix inaccuracies

data["County"].replace("De kalb County", "De Kalb County", inplace=True)
data["County"].replace("Dekalb County", "De Kalb County", inplace=True)
data["County"].replace("Saint clair County", "St. Clair County", inplace=True)
data["County"].replace("Hot spring County", "Hot Spring County", inplace=True)
data["County"].replace("Little river County", "Little River County", inplace=True)
data["County"].replace("Saint francis County", "St. Francis County", inplace=True)
data["County"].replace("Van buren County", "Van Buren County", inplace=True)
data["County"].replace("Desoto County", "De Soto County", inplace=True)
data["County"].replace("De soto County", "De Soto County", inplace=True)
data["County"].replace("Indian river County", "Indian River County", inplace=True)
data["County"].replace("Miami-dade County", "Miami-Dade County", inplace=True)
data["County"].replace("Palm beach County", "Palm Beach County", inplace=True)
data["County"].replace("Saint johns County", "St. Johns County", inplace=True)
data["County"].replace("Saint lucie County", "St. Lucie County", inplace=True)
data["County"].replace("Santa rosa County", "Santa Rosa County", inplace=True)
data["County"].replace("Ben hill County", "Ben Hill County", inplace=True)
data["County"].replace("Jeff davis County", "Jeff Davis County", inplace=True)
data["County"].replace("Pearl river County", "Pearl River County", inplace=True)
data["County"].replace("Mcdowell County", "McDowell County", inplace=True)
data["County"].replace("New hanover County", "New Hanover County", inplace=True)
data["County"].replace("Mccormick County", "McCormick County", inplace=True)
data["County"].replace("Mcduffie County", "McDuffie County", inplace=True)
data["County"].replace("Mcintosh County", "McIntosh County", inplace=True)
data["County"].replace("Jefferson davis County", "Jefferson Davis County", inplace=True)
data["County"].replace("Mcnairy County", "McNairy County", inplace=True)
data["County"].replace("Mcminn County", "McMinn County", inplace=True)
data["County"].replace("Saint joseph County", "St. Joseph County", inplace=True)
data["County"].replace("Presque isle County", "Presque Isle County", inplace=True)
data["County"].replace("Grand traverse County", "Grand Traverse County", inplace=True)
data["County"].replace("Rock island County", "Rock Island County", inplace=True)
data["County"].replace("Mclean County", "McLean County", inplace=True)
data["County"].replace("Mchenry County", "McHenry County", inplace=True)
data["County"].replace("Mcdonough County", "McDonough County", inplace=True)
data["County"].replace("La salle County", "LaSalle County", inplace=True)
data["County"].replace("Jo daviess County", "Jo Daviess County", inplace=True)
data["County"].replace("Dupage County", "Du Page County", inplace=True)
data["County"].replace("Dewitt County", "De Witt County", inplace=True)
data["County"].replace("Black hawk County", "Black Hawk County", inplace=True)
data["County"].replace("Buena vista County", "Buena Vista County", inplace=True)
data["County"].replace("Cerro gordo County", "Cerro Gordo County", inplace=True)
data["County"].replace("Des moines County", "Des Moines County", inplace=True)
data["County"].replace("Obrien County", "O Brien County", inplace=True)
data["County"].replace("Palo alto County", "Palo Alto County", inplace=True)

In [11]:
# merge data and check
merged = pd.merge(
    fips_cut, data, on=["County", "State"], how="outer", validate="1:m", indicator=True
)
merged[merged["_merge"] != "both"]

,FIPS,County,State,State Name,Year,Total_Weight,_merge
1104,13053,Chattahoochee County,GA,Georgia,NaN,NaN,left_only
1266,13101,Echols County,GA,Georgia,NaN,NaN,left_only
1732,13239,Quitman County,GA,Georgia,NaN,NaN,left_only
1812,13265,Taliaferro County,GA,Georgia,NaN,NaN,left_only
1953,13307,Webster County,GA,Georgia,NaN,NaN,left_only
2192,28055,Issaquena County,MS,Mississippi,NaN,NaN,left_only


In [12]:
# check that there's no weird misspellings of the counties or other obvious misaignments
ga_fips = list(fips[fips["State"].isin(["GA"])]["County"].unique())
ga_data = list(data[data["State"].isin(["GA"])]["County"].unique())
print(list(set(ga_fips) - set(ga_data)))

mi_fips = list(fips[fips["State"].isin(["MI"])]["County"].unique())
mi_data = list(data[data["State"].isin(["MI"])]["County"].unique())
print(list(set(mi_fips) - set(mi_data)))

ms_fips = list(fips[fips["State"].isin(["MS"])]["County"].unique())
ms_data = list(data[data["State"].isin(["MS"])]["County"].unique())
print(list(set(ms_fips) - set(ms_data)))

nc_fips = list(fips[fips["State"].isin(["NC"])]["County"].unique())
nc_data = list(data[data["State"].isin(["NC"])]["County"].unique())
print(list(set(nc_fips) - set(nc_data)))

# the remaining counties are not in the original dataset, likely because they had no drug shipments. We are okay omitting these.

['Taliaferro County', 'Webster County', 'Chattahoochee County', 'Quitman County', 'Echols County']
['Grand Traverse County', 'Alpena County', 'Kalamazoo County', 'Gogebic County', 'Calhoun County', 'Washtenaw County', 'Iron County', 'Oakland County', 'Bay County', 'Mecosta County', 'Mason County', 'Montmorency County', 'Menominee County', 'Chippewa County', 'Arenac County', 'Ogemaw County', 'Hillsdale County', 'Clinton County', 'Kent County', 'Iosco County', 'Isabella County', 'Allegan County', 'Missaukee County', 'Crawford County', 'Montcalm County', 'Gratiot County', 'Lenawee County', 'Saginaw County', 'Van Buren County', 'Jackson County', 'Genesee County', 'Alger County', 'Osceola County', 'Macomb County', 'Manistee County', 'Ingham County', 'Ottawa County', 'Presque Isle County', 'Kalkaska County', 'Lapeer County', 'Mackinac County', 'Barry County', 'Ontonagon County', 'Tuscola County', 'Luce County', 'Emmet County', 'Sanilac County', 'St. Clair County', 'Midland County', 'Clare Co

In [13]:
# make final FIPS/drug shipment dataset
drugship = merged[merged["_merge"] == "both"]

assert (drugship._merge == "both").all()

# drop _merge column and reorder
drugship = drugship.drop(columns="_merge")
drugship = drugship[["State", "State Name", "County", "FIPS", "Year", "Total_Weight"]]
drugship.head()

,State,State Name,County,FIPS,Year,Total_Weight
0,AL,Alabama,Autauga County,1001,2006.0,17100.542225
1,AL,Alabama,Autauga County,1001,2007.0,19152.437289
2,AL,Alabama,Autauga County,1001,2008.0,21785.835290
3,AL,Alabama,Autauga County,1001,2009.0,24439.987693
4,AL,Alabama,Autauga County,1001,2010.0,26169.308158


In [14]:
# merge in population data. Use left because not all states are in dataset; want to keep all drugship observations

drug_pop = pd.merge(
    drugship, pop, on=["FIPS", "Year"], how="left", validate="m:m", indicator=True
)
# check
assert (drug_pop._merge == "both").all()

# drop irrelevant columns
drug_pop = drug_pop.drop(
    columns=["County_x", "State Name_x", "Unnamed: 0", "State", "_merge"]
)
# rename colums
drug_pop = drug_pop.rename(columns={"State Name_y": "State", "County_y": "County"})

# reorder columns
drug_pop = drug_pop[
    ["State Abbr", "State", "County", "FIPS", "Year", "Total_Weight", "Population"]
]

In [15]:
# do some reasonableness checks
assert len(drug_pop["State"].unique()) == len(list(data["State"].unique()))
assert (drug_pop["Total_Weight"] > 0).all
assert (drug_pop["Population"] > 0).all

In [16]:
# create drug shipment per capita column
# Create per capita deaths column
drug_pop["PerCapWeight"] = (drug_pop["Total_Weight"] / drug_pop["Population"]) * 100000

In [17]:
# export clean dataset to csv
drug_pop.to_csv(
    "/Users/samsloate/Desktop/Data_Science/Opioids_Project/estimating-impact-of-opioids-2020-purlple-team/20_intermediate_files/drug_shipment_data.csv"
)